In [31]:
import scipy as sp
import scipy.sparse as sps
import numpy as np
import torch

def scipy_sparse_to_torch_sparse(matrix):
    """Converts a scipy.sparse matrix to a PyTorch sparse tensor."""
    if not sps.isspmatrix_coo(matrix):
        matrix = matrix.tocoo()

    values = matrix.data
    indices = np.vstack((matrix.row, matrix.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = matrix.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

EJ = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
EL = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
ECJ = torch.tensor(20, requires_grad=True, dtype=torch.double)
EC = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
dEJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
dCJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
flux = torch.tensor(0.5, requires_grad=True, dtype=torch.double)
ng = 0.1
ncut = 30
truncated_dim = 10
pt_count = 10
min_val = -19
max_val = 19
hamiltonian_creation = 'auto_H'


phi_ext = 0.5
varphi_ext =0.5

Nphi = 100
Ntheta = 100

eye_Nphi = sps.eye(Nphi)
eye_Ntheta = sps.eye(Ntheta)

partial_phi_fd = scipy_sparse_to_torch_sparse(sps.kron(eye_Ntheta, sps.diags([-1, 1, 1], [0, 1, -Nphi+1], shape=(Nphi, Nphi))))

partial_phi_bk = scipy_sparse_to_torch_sparse(sps.kron(eye_Ntheta, sps.diags([1, -1, -1], [0, -1, Nphi-1], shape=(Nphi, Nphi))))

partial_theta_fd = scipy_sparse_to_torch_sparse(sps.kron(eye_Nphi, sps.diags([-1, 1, 1], [0, 1, -Ntheta+1], shape=(Ntheta, Ntheta))))

partial_theta_bk = scipy_sparse_to_torch_sparse(sps.kron(eye_Nphi, sps.diags([1, -1, -1], [0, -1, Ntheta-1], shape=(Ntheta, Ntheta))))


phi = np.linspace(0, 2 * np.pi, Nphi)
cos_phi = np.cos(phi)
cos_phi_m = np.diag(cos_phi)
sin_phi_adj = np.sin(phi-phi_ext/2)
sin_phi_adj_m = np.diag(sin_phi_adj)
phi_m = np.diag(phi)
_cos_phi = torch.kron(torch.tensor(cos_phi_m), torch.tensor(eye_Nphi.todense()) ).to_sparse()
_phi = torch.kron(torch.tensor(phi_m), torch.tensor(eye_Nphi.todense()) ).to_sparse()
_sin_phi_adj_m  = torch.kron(torch.tensor(sin_phi_adj_m), torch.tensor(eye_Nphi.todense())).to_sparse()

theta = np.linspace(0, 2 * np.pi, Ntheta)
cos_theta_adj = np.cos(theta-varphi_ext/2)
sin_theta = np.sin(theta)
cos_theta_adj_m = np.diag(cos_theta_adj)
sin_theta_m = np.diag(sin_theta)
_cos_theta_adj_m = torch.kron(torch.tensor(cos_theta_adj_m), torch.tensor(eye_Ntheta.todense())).to_sparse()
_sin_theta_m = torch.kron(torch.tensor(sin_theta_m), torch.tensor(eye_Ntheta.todense())).to_sparse()

#What do the krons do? The partial derivative operators are identical  - why would we chose different N to make them different? 

print(partial_phi_fd.size())
print(partial_phi_bk.size())
print(partial_theta_fd.size())
print(partial_theta_bk.size())

print(_cos_phi .size())
print(_phi.size())
print(_sin_phi_adj_m.size())
print(_cos_theta_adj_m.size())
print(_sin_theta_m.size())


a = -2 * ECj * (partial_phi_fd * partial_phi_bk) 
b = 2 * ECs * ((1j* partial_theta_fd.to_dense() - ng)**2).to_sparse()
c = 2 * EJ * _cos_phi * _cos_theta_adj_m
d = EL * _phi ** 2
e = 2 * EJ * torch.kron(torch.tensor(eye_Nphi.todense()), torch.tensor(eye_Ntheta.todense())).to_sparse()
f = EJ * dEj * _sin_theta_m * _sin_phi_adj_m
print(a)
print(a.size())
print(b)
print(b.size())
print(c)
print(c.size())
print(d)
print(d.size())
print(e)
print(e.size())
print(f)
print(f.size())

I = torch.kron(torch.tensor(eye_Nphi.todense()), torch.tensor(eye_Ntheta.todense())).to_sparse()

H = -2 * ECj * (partial_phi_fd * partial_phi_bk) \
    + 2 * ECs * (-1* partial_theta_fd**2 +ng**2*I-2*ng*partial_theta_fd)\
    + 2 * ECs * dCj * partial_phi_fd * partial_theta_fd \
    - 2 * EJ * _cos_phi * _cos_theta_adj_m \
    + EL * _phi ** 2 \
    + 2 * EJ * I  \
    + EJ * dEj * _sin_theta_m * _sin_phi_adj_m
H

torch.Size([10000, 10000])
torch.Size([10000, 10000])
torch.Size([10000, 10000])
torch.Size([10000, 10000])
torch.Size([10000, 10000])
torch.Size([10000, 10000])
torch.Size([10000, 10000])
torch.Size([10000, 10000])
torch.Size([10000, 10000])
tensor(indices=tensor([[   0,    1,    2,  ..., 9997, 9998, 9999],
                       [   0,    1,    2,  ..., 9997, 9998, 9999]]),
       values=tensor([10., 10., 10.,  ..., 10., 10., 10.]),
       size=(10000, 10000), nnz=10000, layout=torch.sparse_coo,
       grad_fn=<MulBackward0>)
torch.Size([10000, 10000])
tensor(indices=tensor([[   0,    0,    0,  ..., 9999, 9999, 9999],
                       [   0,    1,    2,  ..., 9997, 9998, 9999]]),
       values=tensor([ 0.+20.j,  0.-20.j, 10.+0.j,  ..., 10.+0.j,
                      10.+0.j,  0.+20.j]),
       size=(10000, 10000), nnz=100000000, layout=torch.sparse_coo,
       grad_fn=<MulBackward0>)
torch.Size([10000, 10000])
tensor(indices=tensor([[   0,    1,    2,  ..., 9997, 9998, 9999],
 

tensor(indices=tensor([[   0,    0,    1,  ..., 9997, 9998, 9999],
                       [   0,    1,    1,  ..., 9998, 9999, 9900]]),
       values=tensor([ 80.3109, -10.0000,  80.3109,  ...,  30.0000,
                       30.0000,  30.0000]),
       size=(10000, 10000), nnz=30000, dtype=torch.float64,
       layout=torch.sparse_coo, grad_fn=<AddBackward0>)

In [47]:
import scipy as sp
sp.linalg.eigh(H.to_dense().detach().numpy())


(array([ 59.86364592,  59.92336691,  60.13384036, ..., 329.33835129,
        333.10479139, 337.18138141]),
 array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        , -0.70710678]]))

In [58]:

x,y  =torch.lobpcg(H.to_dense(), k=10 )

dE = x[0]-x[1]

dE.backward()

EJ.grad

_LinAlgError: linalg.cholesky: The factorization could not be completed because the input is not positive-definite (the leading minor of order 9816 is not positive-definite).

In [ ]:
import DominantSparseEigenAD.symeig as symeig

symeig.setDominantSparseSymeig(A, Aadjoint_to_padjoint)
dominantsparsesymeig = symeig.DominantSparseSymeig.apply

# Usage
dominantsparsesymeig(EJ, 1, dim)

In [13]:
import torch
from jax import grad
import jax.numpy as jnp


# Create a PyTorch tensor
torch_tensor = torch.tensor([1, 2, 3, 4, 5.])

# Convert PyTorch tensor to JAX array
jax_array = jnp.array(torch_tensor.numpy())

print(jax_array)



ModuleNotFoundError: No module named 'jax._src.config'